In [2]:
import numpy as np

In [3]:
def sigmoid(x):
  # Наша функция активации: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

In [4]:
class Neuron:
  def __init__(self, weights, bias):
    self.weights = weights
    self.bias = bias

  def feedforward(self, inputs):
    # Умножаем входы на веса, прибавляем порог, затем используем функцию активации
    total = np.dot(self.weights, inputs) + self.bias
    return sigmoid(total)

In [5]:
weights = np.array([0, 1]) # w1 = 0, w2 = 1
bias = 4                   # b = 4
n = Neuron(weights, bias)

x = np.array([2, 3])       # x1 = 2, x2 = 3
print(n.feedforward(x))    # 0.9990889488055994

0.9990889488055994


In [6]:
class OurNeuralNetwork:
    def __init__(self):
        # Веса
        self.w1 = np.random.normal()
        self.w2 = np.random.normal()
        self.w3 = np.random.normal()
        self.w4 = np.random.normal()
        self.w5 = np.random.normal()
        self.w6 = np.random.normal()

        # Пороги
        self.b1 = np.random.normal()
        self.b2 = np.random.normal()
        self.b3 = np.random.normal()

    def feedforward(self, x):
        # x is a numpy array with 2 elements.
        h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
        h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
        o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
        return o1

    def predict(self, data):
        predictions = []
        for x in data:
            predictions.append(self.feedforward(x))
        return predictions

    def train(self, data, all_y_trues):
        '''
        - data - массив numpy (n x 2) numpy, n = к-во наблюдений в наборе.
        - all_y_trues - массив numpy с n элементами.
          Элементы all_y_trues соответствуют наблюдениям в data.
        '''
        learn_rate = 0.1
        epochs = 1000  # сколько раз пройти по всему набору данных

        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                # --- Прямой проход (эти значения нам понадобятся позже)
                sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
                h1 = sigmoid(sum_h1)

                sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
                h2 = sigmoid(sum_h2)

                sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
                o1 = sigmoid(sum_o1)
                y_pred = o1

                # --- Считаем частные производные.
                # --- Имена: d_L_d_w1 = "частная производная L по w1"
                d_L_d_ypred = -2 * (y_true - y_pred)

                # Нейрон o1
                d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
                d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
                d_ypred_d_b3 = deriv_sigmoid(sum_o1)

                d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
                d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

                # Нейрон h1
                d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
                d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
                d_h1_d_b1 = deriv_sigmoid(sum_h1)

                # Нейрон h2
                d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
                d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
                d_h2_d_b2 = deriv_sigmoid(sum_h2)

                # --- Обновляем веса и пороги
                # Нейрон h1
                self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
                self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

                # Нейрон h2
                self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
                self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

                # Нейрон o1
                self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
                self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
                self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

            # --- Считаем полные потери в конце каждой эпохи
            if epoch % 100 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = mse_loss(all_y_trues, y_preds)
                print("Epoch %d loss: %.3f" % (epoch, loss))

In [1]:
def mse_loss(y_true, y_pred):
  # y_true и y_pred - массивы numpy одинаковой длины.
  return ((y_true - y_pred) ** 2).mean()

In [7]:
def deriv_sigmoid(x):
  # Производная сигмоиды: f'(x) = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx * (1 - fx)

In [11]:
from sklearn.neural_network import MLPClassifier

# Определим набор данных
data = np.array([
    [-2, -1],  # Алиса
    [25, 6],   # Боб
    [17, 4],   # Чарли
    [-15, -6], # Диана
])
all_y_trues = np.array([
    1, # Алиса
    0, # Боб
    0, # Чарли
    1, # Диана
])



In [12]:
# Создаем и обучаем нейросеть
classifier = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)
classifier.fit(data, all_y_trues)

MLPClassifier(max_iter=1000)

In [13]:
# Оценим точность классификатора
accuracy = classifier.score(data, all_y_trues)
print("Accuracy:", accuracy)

Accuracy: 1.0


In [14]:
network = OurNeuralNetwork()
network.train(data, all_y_trues)

Epoch 0 loss: 0.312
Epoch 100 loss: 0.023
Epoch 200 loss: 0.010
Epoch 300 loss: 0.006
Epoch 400 loss: 0.004
Epoch 500 loss: 0.003
Epoch 600 loss: 0.003
Epoch 700 loss: 0.002
Epoch 800 loss: 0.002
Epoch 900 loss: 0.002


In [15]:
# Прогнозирование и оценка точности
predictions = network.predict(data)
rounded_predictions = [round(p) for p in predictions]
accuracy = np.mean(rounded_predictions == all_y_trues)
print("Accuracy:", accuracy)

Accuracy: 1.0


Этот код обучает нейронную сеть на данных а затем применяет метод predict, чтобы сделать прогнозы и оценить точность с использованием предоставленных меток.

Улучшение точности нейронной сети может зависеть от многих факторов, включая архитектуру сети, параметры обучения и характеристики данных. Рассмотрим некоторые из них:

Что может привести к ухудшению точности нейронной сети:

1.Слишком простая архитектура: Если нейронная сеть слишком простая, она может не иметь достаточной емкости для моделирования сложных зависимостей в данных.

2.Переобучение: Это происходит, когда модель слишком хорошо подстроена под обучающие данные, включая шум и ненужные детали, что делает её менее обобщающей на новые данные.

3.Недообучение: Наоборот, это происходит, когда модель недостаточно обучена на тренировочных данных и не смогла захватить закономерности в данных.

4.Сложность архитектуры: Слишком сложные модели могут стать сложными для обучения и подвержены переобучению.

Что может привести к увеличению точности нейронной сети:

1.Добавление данных: Больше данных для обучения может помочь сети лучше обобщить зависимости и избежать переобучения.

2.Регуляризация: Использование методов регуляризации, таких как L1 или L2 регуляризация, может помочь предотвратить переобучение.

3.Выбор архитектуры: Выбор правильной архитектуры сети, включая количество слоев, количество нейронов и функции активации, может значительно повлиять на её способность обучения и обобщения.

4.Настройка гиперпараметров: Экспериментирование с гиперпараметрами, такими как скорость обучения, количество эпох и размер мини-партии, может помочь найти оптимальную конфигурацию для обучения сети.

5.Использование предварительно обученных моделей: В случае недостаточного количества данных для обучения можно воспользоваться предварительно обученными моделями, которые обучены на больших наборах данных и затем дообучить их на своих данных.

Это лишь некоторые из факторов, которые могут влиять на точность нейронных сетей. Оптимизация процесса обучения и выбор подходящей архитектуры сети обычно требует экспериментов и анализа результатов.